In [74]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config('spark.app.name', 'itv012041_Loan_Repayment_Data_Cleanup'). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [75]:
spark

In [76]:
loans_repay_schema = """loan_id string,total_principal_received float,total_interest_received
float,total_late_fee_received float,total_payment_received
float,last_payment_amount float,last_payment_date string,next_payment_date
string"""

In [77]:
loans_repay_raw_df = spark.read \
.format("csv") \
.option("header", True) \
.schema(loans_repay_schema) \
.load("/user/itv012041/lendingclubproject/raw/loans_repayments_data_csv")

In [78]:
loans_repay_raw_df

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date
491699,7000.0,1444.94,0.0,8444.942,954.65,Dec-2012,null
491685,15000.0,2583.58,0.0,17583.584,517.23,Mar-2013,null
491667,6400.0,807.51,0.0,7207.5093,209.97,Mar-2013,null
491160,4000.0,963.31,0.0,4963.3066,144.87,Mar-2013,null
491675,20000.0,2887.0,0.0,22886.992,16.27,Jul-2011,null
491668,1773.57,591.33,0.0,2957.37,394.94,Mar-2011,null
491663,5500.0,235.4,0.0,5735.4,2147.02,Sep-2010,null
491632,10000.0,1466.31,0.0,11466.3125,7274.73,Apr-2011,null
491618,24999.99,2085.75,0.0,27085.738,21866.75,Oct-2010,null
491622,25000.0,13079.81,0.0,38079.81,7912.97,Feb-2015,null


In [79]:
loans_repay_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- total_principal_received: float (nullable = true)
 |-- total_interest_received: float (nullable = true)
 |-- total_late_fee_received: float (nullable = true)
 |-- total_payment_received: float (nullable = true)
 |-- last_payment_amount: float (nullable = true)
 |-- last_payment_date: string (nullable = true)
 |-- next_payment_date: string (nullable = true)



In [80]:
from pyspark.sql.functions import current_timestamp

In [81]:
loans_repay_df_with_ingest_date = loans_repay_raw_df.withColumn("ingest_date", current_timestamp())

In [82]:
loans_repay_df_with_ingest_date

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
491699,7000.0,1444.94,0.0,8444.942,954.65,Dec-2012,null,2024-12-20 04:33:...
491685,15000.0,2583.58,0.0,17583.584,517.23,Mar-2013,null,2024-12-20 04:33:...
491667,6400.0,807.51,0.0,7207.5093,209.97,Mar-2013,null,2024-12-20 04:33:...
491160,4000.0,963.31,0.0,4963.3066,144.87,Mar-2013,null,2024-12-20 04:33:...
491675,20000.0,2887.0,0.0,22886.992,16.27,Jul-2011,null,2024-12-20 04:33:...
491668,1773.57,591.33,0.0,2957.37,394.94,Mar-2011,null,2024-12-20 04:33:...
491663,5500.0,235.4,0.0,5735.4,2147.02,Sep-2010,null,2024-12-20 04:33:...
491632,10000.0,1466.31,0.0,11466.3125,7274.73,Apr-2011,null,2024-12-20 04:33:...
491618,24999.99,2085.75,0.0,27085.738,21866.75,Oct-2010,null,2024-12-20 04:33:...
491622,25000.0,13079.81,0.0,38079.81,7912.97,Feb-2015,null,2024-12-20 04:33:...


In [83]:
loans_repay_df_with_ingest_date.createOrReplaceTempView("loans_repayments")

In [84]:
spark.sql("select * from loans_repayments")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
491699,7000.0,1444.94,0.0,8444.942,954.65,Dec-2012,null,2024-12-20 04:33:...
491685,15000.0,2583.58,0.0,17583.584,517.23,Mar-2013,null,2024-12-20 04:33:...
491667,6400.0,807.51,0.0,7207.5093,209.97,Mar-2013,null,2024-12-20 04:33:...
491160,4000.0,963.31,0.0,4963.3066,144.87,Mar-2013,null,2024-12-20 04:33:...
491675,20000.0,2887.0,0.0,22886.992,16.27,Jul-2011,null,2024-12-20 04:33:...
491668,1773.57,591.33,0.0,2957.37,394.94,Mar-2011,null,2024-12-20 04:33:...
491663,5500.0,235.4,0.0,5735.4,2147.02,Sep-2010,null,2024-12-20 04:33:...
491632,10000.0,1466.31,0.0,11466.3125,7274.73,Apr-2011,null,2024-12-20 04:33:...
491618,24999.99,2085.75,0.0,27085.738,21866.75,Oct-2010,null,2024-12-20 04:33:...
491622,25000.0,13079.81,0.0,38079.81,7912.97,Feb-2015,null,2024-12-20 04:33:...


In [85]:
spark.sql("select count(*) from loans_repayments where total_principal_received is null")

count(1)
69


In [86]:
columns_to_check = ["total_principal_received","total_interest_received","total_late_fee_received",
                    "total_payment_received","last_payment_amount"]

In [87]:
loans_repay_filtered_df = loans_repay_df_with_ingest_date.na.drop(subset = columns_to_check)

In [88]:
loans_repay_filtered_df.createOrReplaceTempView("loans_repayments")

In [89]:
loans_repay_df_with_ingest_date.count()

2260701

In [90]:
loans_repay_filtered_df.count()

2260498

In [91]:
spark.sql("select * from loans_repayments where total_payment_received = 0.0")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
485818,14640.096,13388.84,13000.0,0.0,0.0,0.0,Mar-2013,2024-12-20 04:33:...
485471,29620.818,29134.64,25000.0,0.0,0.0,0.0,Mar-2013,2024-12-20 04:33:...
482256,8735.611,7479.87,8000.0,0.0,0.0,0.0,Feb-2011,2024-12-20 04:33:...
478160,410.0,407.36,0.0,0.0,143.1,410.0,null,2024-12-20 04:33:...
476557,28865.18,24164.67,5692.31,0.0,6972.59,19916.78,Dec-2010,2024-12-20 04:33:...
472516,25951.482,24731.76,25000.0,0.0,0.0,0.0,May-2010,2024-12-20 04:33:...
472197,12048.13,12018.01,10000.0,0.0,0.0,0.0,Jan-2013,2024-12-20 04:33:...
467364,29216.791,29066.19,24250.0,0.0,0.0,0.0,Dec-2012,2024-12-20 04:33:...
399499,26557.729,26336.41,24000.0,0.0,0.0,0.0,Dec-2010,2024-12-20 04:33:...
451482,7587.5513,7587.55,7000.0,0.0,0.0,0.0,Jan-2011,2024-12-20 04:33:...


In [92]:
spark.sql("select count(*) from loans_repayments where total_payment_received = 0.0")

count(1)
995


In [93]:
spark.sql("select count(*) from loans_repayments \
          where total_payment_received = 0.0 and total_principal_received != 0.0")

count(1)
46


In [94]:
spark.sql("select * from loans_repayments \
          where total_payment_received = 0.0 and total_principal_received != 0.0")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
485818,14640.096,13388.84,13000.0,0.0,0.0,0.0,Mar-2013,2024-12-20 04:33:...
485471,29620.818,29134.64,25000.0,0.0,0.0,0.0,Mar-2013,2024-12-20 04:33:...
482256,8735.611,7479.87,8000.0,0.0,0.0,0.0,Feb-2011,2024-12-20 04:33:...
478160,410.0,407.36,0.0,0.0,143.1,410.0,null,2024-12-20 04:33:...
476557,28865.18,24164.67,5692.31,0.0,6972.59,19916.78,Dec-2010,2024-12-20 04:33:...
472516,25951.482,24731.76,25000.0,0.0,0.0,0.0,May-2010,2024-12-20 04:33:...
472197,12048.13,12018.01,10000.0,0.0,0.0,0.0,Jan-2013,2024-12-20 04:33:...
467364,29216.791,29066.19,24250.0,0.0,0.0,0.0,Dec-2012,2024-12-20 04:33:...
399499,26557.729,26336.41,24000.0,0.0,0.0,0.0,Dec-2010,2024-12-20 04:33:...
451482,7587.5513,7587.55,7000.0,0.0,0.0,0.0,Jan-2011,2024-12-20 04:33:...


In [95]:
from pyspark.sql.functions import when, col

In [96]:
loans_payments_fixed_df = loans_repay_filtered_df.withColumn(
    "total_payment_received",
    when(
        (col("total_principal_received") != 0.0) &
        (col("total_payment_received") == 0.0),
        col("total_principal_received")+col("total_interest_received")+col("total_late_fee_received")
    ).otherwise(col("total_payment_received"))
)

In [97]:
loans_payments_fixed_df

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
491699,7000.0,1444.94,0.0,8444.942,954.65,Dec-2012,null,2024-12-20 04:33:...
491685,15000.0,2583.58,0.0,17583.584,517.23,Mar-2013,null,2024-12-20 04:33:...
491667,6400.0,807.51,0.0,7207.5093,209.97,Mar-2013,null,2024-12-20 04:33:...
491160,4000.0,963.31,0.0,4963.3066,144.87,Mar-2013,null,2024-12-20 04:33:...
491675,20000.0,2887.0,0.0,22886.992,16.27,Jul-2011,null,2024-12-20 04:33:...
491668,1773.57,591.33,0.0,2957.37,394.94,Mar-2011,null,2024-12-20 04:33:...
491663,5500.0,235.4,0.0,5735.4,2147.02,Sep-2010,null,2024-12-20 04:33:...
491632,10000.0,1466.31,0.0,11466.3125,7274.73,Apr-2011,null,2024-12-20 04:33:...
491618,24999.99,2085.75,0.0,27085.738,21866.75,Oct-2010,null,2024-12-20 04:33:...
491622,25000.0,13079.81,0.0,38079.81,7912.97,Feb-2015,null,2024-12-20 04:33:...


In [98]:
loans_payments_fixed_df.filter("loan_id == '485818'")

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
485818,14640.096,13388.84,13000.0,41028.938,0.0,0.0,Mar-2013,2024-12-20 04:33:...


In [99]:
loans_payments_fixed_df.filter("total_payment_received = 0.0").count()

949

In [100]:
loans_payments_fixed_df2 = loans_payments_fixed_df.filter("total_payment_received = 0.0")

In [101]:
loans_payments_fixed_df2

loan_id,total_principal_received,total_interest_received,total_late_fee_received,total_payment_received,last_payment_amount,last_payment_date,next_payment_date,ingest_date
454682,0.0,0.0,0.0,0.0,0.0,null,null,2024-12-20 04:33:...
451820,0.0,0.0,0.0,0.0,0.0,null,null,2024-12-20 04:33:...
439641,0.0,0.0,0.0,0.0,0.0,null,null,2024-12-20 04:33:...
417817,0.0,0.0,0.0,0.0,0.0,null,null,2024-12-20 04:33:...
392751,0.0,0.0,0.0,0.0,0.0,null,null,2024-12-20 04:33:...
472762,0.0,0.0,0.0,0.0,0.0,null,Jun-2010,2024-12-20 04:33:...
462167,0.0,0.0,0.0,0.0,0.0,null,May-2010,2024-12-20 04:33:...
409248,0.0,0.0,0.0,0.0,0.0,null,Nov-2009,2024-12-20 04:33:...
271819,0.0,0.0,0.0,0.0,0.0,null,Dec-2008,2024-12-20 04:33:...
103567719,0.0,0.0,0.0,0.0,0.0,null,null,2024-12-20 04:33:...


In [122]:
spark

In [124]:
loans_payments_fixed_df2.write \
.option("header", True) \
.format("csv") \
.mode("overwrite") \
.option("path", "/user/itv012041/lendingclubproject/cleaned/loans_repayments_csv") \
.save()

In [125]:
loans_payments_fixed_df2.write \
.option("header", True) \
.format("parquet") \
.mode("overwrite") \
.option("path", "/user/itv012041/lendingclubproject/cleaned/loans_repayments_parquet") \
.save()